In [6]:
aux = []
for i in range(3):
    aux.append(i)

In [9]:
estados = list(product(*[aux for i in range(2)]))

In [16]:
(2**10)**2

1048576

In [10]:
estados

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [54]:
import numpy as np
import random
from itertools import product
import pandas as pd


class Frota:
    def __init__(self, Periodo, TamanhoFrota, Demanda):
        if len(Demanda) < Periodo:
            Demanda = Demanda + \
                [random.randint(0, TamanhoFrota)
                 for i in range(Periodo-len(Demanda))]
        self.ProbQuebra_0 = 0.01
        self.Depreciacao = 0.4
        self.ProbPar = 0.4
        # self.Parametros = {}
        self.Periodo = Periodo
        self.TamanhoFrota = TamanhoFrota
        self.Demanda = Demanda
        self.ProbQuebraFcn = lambda x: self.ProbQuebra_0 / \
            (self.ProbQuebra_0+(1-self.ProbQuebra_0)*np.exp(-self.ProbPar*x))
        self.Politica = {}
       
    def AcoesDisponiveis(self,estado):
        acoes = list(product(*[[0, 1] for i in range(self.TamanhoFrota)]))
        aux = []
        for ac in acoes:
            cnt = sum(ac)
            print(cnt)
            if estado[1]>=self.Periodo:
                return {estado:[tuple([-1]*self.TamanhoFrota)]} 
            if cnt >= self.Demanda[estado[1]]:
                aux.append(ac)
            
            
        return {estado:aux}

    def Recompensa(self, est):
        tvida = 0

        # define o valor do equipamento como função da idade do equipamento
        valor_equipamento = np.exp(-0.005*tvida)

        # define o valor da manutenção
        valor_manutencao = 0.02 - 0.01*np.exp(-0.1*tvida)

        # define o valor da manutenção se der defeito
        valor_defeito = 5*valor_manutencao/valor_equipamento

        # Cálculo das recompensas SEM defeito e COM defeito
        res_a0_op, res_a0_de = 0, valor_defeito

        # Cálculo das recompensas na manutenção
        res_a1 = valor_manutencao/valor_equipamento

        
        res = [res_a0_op, res_a0_de, res_a1]
        if est[0] == -1:
            res = [0, 0, 0]
        return res

    def ProximoEstado(self, estado, acao):
        NovosEstados = {}
        NovoEstado = [[]]*self.TamanhoFrota
        est,tmp = estado[0],estado[1]
        for i, a in enumerate(acao):
            rcp = self.Recompensa(est[i])
            prob_quebrar = self.ProbQuebraFcn(estado[i])
            if a == 0:
                NovoEstado[i] = [(est[i]+1,rcp[0],1-prob_quebrar),(0,rcp[1],prob_quebrar)]
            if a == 1:
                NovoEstado[i] = [(0,rcp[2],1)]
            if a == -1:
                NovoEstado[i] = [(-1,0,1)]
                
        # print(NovoEstado)

        estaux = list(product(*NovoEstado))
        # recs = list(product(*Recompensa))
        # probs = [np.prod(pr) for pr in probs]
        # recs = [np.sum(rc) for rc in recs]
        # if self.Periodo > estado[-1]:
        #     tmp = estado[-1]+1
        # else:
        #     tmp = self.Periodo

        # NovosEstados["estados"] = [(*x, tmp)
        #                            for x in list(product(*NovoEstado))]
        # NovosEstados["recompensas"] = recs
        # NovosEstados["probabilidades"] = probs

        return estaux

    # def ValueIteration(self, tol=1e-5, sims=500):

    #     lista_estados = list(self.EstadosAcoes.keys())
    #     # est0 = lista_estados[0]
    #     if len(self.Politica) == 0:
    #         for est in lista_estados:
    #             aux = [0]*len(self.EstadosAcoes[est])
    #             # aux[random.choice(range(len(aux)))] = 1
    #             aux[0] = 1
    #             self.Politica[est] = {ac: x for ac,x in zip(self.EstadosAcoes[est], aux)}

    #     VN = {v: 0.05 for v in lista_estados}
    #     VN[(*tuple([(-1, -1)]*self.TamanhoFrota), self.Periodo)] = 0

    #     PLN = {k: np.argmax(v) for k, v in self.Politica.items()}

    #     M, P = {}, {}
    #     M[0] = np.array(list(VN.values()))
    #     P[0] = {k: v for k, v in PLN.items()}

    #     cnt, nor = 0, 10
    #     val_norm = []
    #     while nor > tol and cnt <= sims:

    #         # PL = {k:v for k,v in PLN.items()}
    #         PLN = {}

    #         V = {k: v for k, v in VN.items()}
    #         VN = {}

    #         for est in lista_estados:
    #             v_aux = []
    #             for idx, ac in enumerate(self.EstadosAcoes[est]):
    #                 next = self.ProximoEstado(est, ac)
    #                 nest = next["estados"]
    #                 probs = next["probabilidades"]
    #                 recs = next["recompensas"]
    #                 v_aux2 = 0
    #                 for j_e in range(len(nest)):
    #                     v_aux2 = v_aux2 + probs[j_e]*(recs[j_e]+V[nest[j_e]])
    #                 v_aux.append(v_aux2)

    #             VN[est] = np.max(v_aux)
    #             PLN[est] = np.argmax(v_aux)

    #         cnt = cnt+1
    #         nor = np.linalg.norm(
    #             np.array(list(V.values()))-np.array(list(VN.values())))
    #         val_norm.append(nor)
    #         print(
    #             f"sim: {cnt:2d}, norma: {nor:5.2f}, condicao: {nor>tol}", end='\r')
    #         M[cnt % 2] = np.array(list(VN.values()))
    #         P[cnt % 2] = {k: v for k, v in PLN.items()}

    #     # PoliticaOtima = {est:self.EstadosAcoes[est][idx] for est,idx in P[1].items()}
    #     self.Politica = {
    #         est: self.EstadosAcoes[est][idx] for est, idx in P[1].items()}

    #     return {"politica": self.Politica, "valor": M[1], "norma": val_norm}

In [59]:
ft = Frota(3,2,[0,2,0])

In [61]:
ft.AcoesDisponiveis(((0,0),1))

0
1
1
2


{((0, 0), 1): [(-1, -1), (-1, -1), (-1, -1), (1, 1)]}